In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pprint
sns.set(style="white", context='poster')
from sympy import *
import math
init_printing(use_unicode=True)
from scipy.integrate import solve_ivp

In [2]:
from numba import njit

In [3]:
import numba

levenberg optimization

In [4]:
%load_ext autoreload
%autoreload 2


In [6]:
#from model_equations_separate_NC import *
from model_equations_separate_NC_store_numba import *


In [7]:

HCOLOR = '#ffc58f'
HCOLOR_light = '#ffd2a9'
PCOLOR = '#99cc99'
PCOLOR_light = '#aad5aa'

DONCOLOR = '#7285b7'
RDONCOLOR = 'black' #'#00334D'
DINCOLOR = '#bbdaff' #'#000066'

DOCCOLOR = '#cc6666'
RDOCCOLOR = 'black' #'#00334D'
DICCOLOR = '#ff9da4' #'#000066'

SHCOLOR = '#ffa98f'
SPCOLOR = '#326232'


ccnpalette = [PCOLOR, PCOLOR_light, HCOLOR, HCOLOR_light, SPCOLOR, SHCOLOR,DONCOLOR, RDONCOLOR, DINCOLOR]
ccnorder = ['Bp', 'Np', 'Bh','Nh',  'ABp', 'ABh', 'DON', 'RDON', 'DIN', ]
cccpalette = [PCOLOR, PCOLOR_light, HCOLOR, HCOLOR_light, SPCOLOR, SHCOLOR, DOCCOLOR, RDOCCOLOR, DICCOLOR]
cccorder = ['Bp[C]', 'Cp', 'Bh[C]', 'Ch', 'ABp[C]', 'ABh[C]', 'DOC', 'RDOC', 'DIC']
ccxpalette = [PCOLOR, HCOLOR]
ccxorder = ['Xp', 'Xh']
hnpalette = [ HCOLOR_light, SPCOLOR, SHCOLOR, DONCOLOR, RDONCOLOR, DINCOLOR]
hnorder = [ 'Bh', 'ABp', 'ABh', 'DON', 'RDON', 'DIN']
hcpalette = [ HCOLOR_light, SPCOLOR, SHCOLOR, DOCCOLOR, RDOCCOLOR, DICCOLOR]
hcorder = [ 'Bh[C]', 'ABp[C]', 'ABh[C]', 'DOC', 'RDOC', 'DIC']
pnpalette = [ PCOLOR_light, SPCOLOR, SHCOLOR, DONCOLOR, RDONCOLOR, DINCOLOR]
pnorder = [ 'Bp', 'ABp', 'ABh', 'DON', 'RDON', 'DIN']
pcpalette = [ PCOLOR_light, SPCOLOR, SHCOLOR, DOCCOLOR, RDOCCOLOR, DICCOLOR]
pcorder = [ 'Bp[C]', 'ABp[C]', 'ABh[C]', 'DOC', 'RDOC', 'DIC']
sns.color_palette(ccnpalette + cccpalette, )
# 


In [8]:

 
sns.color_palette([PCOLOR, HCOLOR, PCOLOR_light, HCOLOR_light] )


In [9]:
refdf = pd.read_excel('reference_10cc_axenic.xlsx')


In [10]:
refdf.columns

Index(['day', 'Sample', 'id', 'replicate', 'transfer', 'transfer day',
       'heterotroph', 'full name', 'FL', 'Group', 'predicted', 'ref_Bp', 'LOD',
       't'],
      dtype='object')

In [13]:
    model='MIN'
    json =[]
    dpath = 'tmp'
    pro99_mode = False
    which_organism = 'ponly'
    t_eval = None
    maxday = 140
    if dpath != '':
        os.makedirs(dpath, exist_ok=True)

    ref_csv= 'reference_10cc_axenic.xlsx'
    refdf = pd.read_excel(ref_csv)
    refdf['ref_Bp'] = refdf['ref_Bp'].clip(lower=4)


    new_param_vals = get_param_vals_from_json_list(model, json)
    suffix = get_runid_unique_suffix(pro99_mode, which_organism, model, new_param_vals)
    t_eval, t_end = get_t_eval_and_t_end(t_eval, refdf, maxday)
    (var_names, init_var_vals, intermediate_names, calc_dydt, prepare_params_tuple
        ) = get_constants_per_organism(pro99_mode, which_organism)


In [15]:
        run_id='tmp'
        run_id = f'{run_id}{suffix}'
        MSE_err = run_solver_from_new_params_and_save(
            new_param_vals, refdf, dpath, 
            run_id, init_var_vals, 
            calc_dydt, prepare_params_tuple, t_end , t_eval, var_names, intermediate_names,
        )

        print ('MSE:', MSE_err)


MSE: 170.04610123765713


In [22]:
        param_sensitivity =2
        organism_to_tune = 'PRO'
        number_of_runs = 5
        parameter, sensitivity_values = get_param_sensitivity_values(
            param_sensitivity, model, organism_to_tune, number_of_runs)


In [23]:
sensitivity_values

array([2.31481481e-07, 5.17608328e-07, 1.15740741e-06, 2.58804164e-06,
       5.78703704e-06])

In [25]:
        for i,v in enumerate(sensitivity_values):
            new_param_vals[parameter] = v
            run_id='tmp'
            run_id = f"{run_id}_{parameter}_{i}_{v}{suffix}"
            print(run_id)
            
            MSE_err = run_solver_from_new_params_and_save(
                new_param_vals, refdf, dpath, 
                run_id, init_var_vals, 
                calc_dydt, prepare_params_tuple, t_end , t_eval, var_names, intermediate_names,
            )
            print ('MSE:', MSE_err)


tmp_E_leakp_0_2.314814814814816e-07_lowN_ponly_MIN_h-1553020579808776996
MSE: 117.94991568659184
tmp_E_leakp_1_5.176083281249514e-07_lowN_ponly_MIN_h-1553020579808776996
MSE: 128.0988664441089
tmp_E_leakp_2_1.1574074074074072e-06_lowN_ponly_MIN_h-1553020579808776996
MSE: 170.04610123780535
tmp_E_leakp_3_2.588041640624755e-06_lowN_ponly_MIN_h-1553020579808776996
MSE: 227.87269120146124
tmp_E_leakp_4_5.787037037037031e-06_lowN_ponly_MIN_h-1553020579808776996
MSE: 260.57111990580785


In [26]:
from run_shgo_ponly import *

In [27]:
os.listdir()

['.ipynb_checkpoints',
 'calc_csat.py',
 'model_equations_separate_NC_store.py',
 'model_equations_separate_NC_store_numba.py',
 'Model_Parameters Store model.xlsx',
 'model_store_cc debug.ipynb',
 'model_store_cc.ipynb',
 'model_store_ponly.ipynb',
 'reference_10cc.xlsx',
 'reference_10cc_axenic.xlsx',
 'reference_pro99_axenic.xlsx',
 'results_postprocess.py',
 'run_files',
 'run_least_squares_het.py',
 'run_least_squares_ponly.py',
 'run_monte.py',
 'run_shgo_ponly.py',
 'tmp',
 '__pycache__']

In [32]:

    out_dpath = dpath
    ref_fpath =  'reference_10cc_axenic.xlsx'
    ref_pro99_fpath =  'reference_pro99_axenic.xlsx'
    json_list = []

In [38]:
    ref_df = pd.read_excel(ref_fpath)
    ref_pro99_df = pd.read_excel(ref_pro99_fpath)
    ref_df = ref_df.sort_values(['t','Sample'])
    ref_pro99_df = ref_pro99_df.sort_values(['t','Sample'])

    Y = pd.concat([ref_df['ref_Bp'], ref_pro99_df['ref_Bp']]).to_numpy()
    Y = np.clip(Y, a_min=4, a_max=None)

    new_param_vals = get_param_vals_from_json_list(model, json_list)
    #TODO
    t_eval, t_end = get_t_eval_and_t_end(None, refdf, maxday=140)
    t_eval_pro99, t_end_pro99 = get_t_eval_and_t_end(None, ref_pro99_df, maxday=140)
    (var_names, init_var_vals, intermediate_names, calc_dydt, prepare_params_tuple
        ) = get_constants_per_organism(False, which_organism)
    (_, init_var_pro99_vals, _, _, _) = get_constants_per_organism(True, which_organism)


    params_to_update, bounds, log_params = get_param_tuning_values(model, organism_to_tune)

    # start with the defalt params
    x0 = np.array([np.log(param_vals[i]) if lg else param_vals[i] for i, lg in zip(params_to_update, log_params)])

    bounds_logged = [(np.log(b[0]),  np.log(b[1]))  if lg else b for b,lg in zip(bounds, log_params)]
    param_bounds =  bounds_logged

    # print('shgo(',
        # fun, param_bounds, 
        # #minimizer_kwargs=dict(
            # #jac=jac_for_minimize, 
    # #        hess=hess_for_minimize, 
    # #        options=dict(disp=True, maxiter=3), method='L-BFGS-B'),
    # #    options=dict(jac=
                # jac_for_minimize)

    additional_params = (
        Y, params_to_update, new_param_vals, log_params, 
        calc_dydt, prepare_params_tuple, var_names, 
        init_var_vals, t_end , t_eval, ref_df, 
        init_var_pro99_vals, t_end_pro99 , t_eval_pro99, ref_pro99_df
    ) 


In [40]:
fun(x0, additional_params)

In [42]:
j =jac(x0, additional_params)

In [43]:
np.max(j)

In [44]:
j

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [45]:
%timeit -n 2 -r 2 fun(x0, additional_params)

4.31 s ± 39.7 ms per loop (mean ± std. dev. of 2 runs, 2 loops each)


In [ ]:
    number_of_runs = 100
    res = shgo(
        fun, param_bounds, args=(additional_params,), 
        n=number_of_runs, iters=1, sampling_method='sobol',
        minimizer_kwargs=dict(
            #jac=jac_for_minimize, 
            hess=hess_for_minimize, 
            options=dict(disp=True, maxiter=3), method='L-BFGS-B'),
        options=dict(jac=jac_for_minimize,disp=True),
    )


Splitting first generation
Generating sampling points
